In [1]:
import cv2
import joblib
import numpy as np
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops  
    

In [2]:
# LOADING SAVED COMPONENTS
print("Loading saved model and components...")
model = joblib.load('xgboost_model.joblib')
label_encoder = joblib.load('label_encoder.joblib')
scaler = joblib.load('feature_scaler.joblib')
print("Loading complete.")

Loading saved model and components...


FileNotFoundError: [Errno 2] No such file or directory: 'xgboost_model.joblib'

In [ ]:
# DEFINING THE SAME FEATURE EXTRACTION METHODS FOR FURTHER CUSTOM IMAGE PREDICTIONS
IMAGE_SIZE = (256, 256)
LBP_RADIUS = 1
LBP_POINTS = 8 * LBP_RADIUS
LBP_METHOD = 'uniform'

In [ ]:
# EXTRACT COLOR, TEXTURE, AND SHAPE FEATURES

def extract_color_features(image):
    
    features = []
    for i in range(3):
        features.extend([np.mean(image[:, :, i]), np.std(image[:, :, i])])
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    for i in range(3):
        features.extend([np.mean(hsv[:, :, i]), np.std(hsv[:, :, i])])
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    features.extend(hist.flatten())
    return features

def extract_texture_features(image):
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, LBP_POINTS, LBP_RADIUS, method=LBP_METHOD)
    (lbp_hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, LBP_POINTS + 3), range=(0, LBP_POINTS + 2))
    lbp_hist = lbp_hist.astype("float") / (lbp_hist.sum() + 1e-7)
    glcm = graycomatrix(gray, distances=[1, 3, 5], angles=[0, np.pi/4, np.pi/2], levels=256, symmetric=True, normed=True)
    glcm_props = [np.mean(graycoprops(glcm, prop)) for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']]
    return np.hstack([lbp_hist, glcm_props]).tolist()

def extract_shape_features(image):
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours: return [0, 0, 0]
    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    aspect_ratio = float(w) / h if h != 0 else 0
    area = cv2.contourArea(c)
    rect_area = w * h
    extent = float(area) / rect_area if rect_area != 0 else 0
    hull = cv2.convexHull(c)
    hull_area = cv2.contourArea(hull)
    solidity = float(area) / hull_area if hull_area != 0 else 0
    return [aspect_ratio, extent, solidity]

In [ ]:
def predict_custom_image(image_path):
    
    
    image = cv2.imread(image_path)
    if image is None:
        return "Error: Image not found or cannot be read."
    image_resized = cv2.resize(image, IMAGE_SIZE)

    # EXTRACT ALL FEATURES INCLUDING COLOR, TEXTURE AND SHAPE
    color_feats = extract_color_features(image_resized)
    texture_feats = extract_texture_features(image_resized)
    shape_feats = extract_shape_features(image_resized)
    
    # COMBINING ALL FEATURES
    features = color_feats + texture_feats + shape_feats
    
    features_reshaped = np.array(features).reshape(1, -1)
    features_scaled = scaler.transform(features_reshaped)
    
    # PREDICT THE OUTPUT FOR THE GIVEN IMAGE
    prediction_numeric = model.predict(features_scaled)
    prediction_label = label_encoder.inverse_transform(prediction_numeric)[0]
    
    return prediction_label

In [ ]:
custom_image_path = '/content/tomato_mosaic_virus.JPG' 
    
predicted_disease = predict_custom_image(custom_image_path)
print(f"The predicted disease for the image is: {predicted_disease}")